In [ ]:
!pip install numpy pandas scikit-learn tqdm scipy openai

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
import scipy as sp
from concurrent.futures import ThreadPoolExecutor

In [ ]:
! pip install -U LibRecommender

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
books = pd.read_csv('/content/drive/MyDrive/Books_Extended.csv', delimiter=';', low_memory=False)
ratings = pd.read_csv('/content/drive/MyDrive/Ratings.csv', delimiter=';')
users = pd.read_csv('/content/drive/MyDrive/Users.csv', delimiter=';')

<ipython-input-7-965afd468f24>:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv('/content/drive/MyDrive/Users.csv', delimiter=';')


## Data processing

In [ ]:
df_prep_step_1 = pd.merge(books, ratings.query("`Rating` > 0"), on='ISBN', how='inner')

# join users data
df_prep_step_2 = pd.merge(df_prep_step_1, users, on='User-ID', how='inner')

In [ ]:
df_prep = df_prep_step_2.drop(['Publisher'], axis=1)
df_isbn = df_prep.drop_duplicates()

In [ ]:
df_prep = df_prep_step_2.drop(['Publisher'], axis=1)

In [ ]:
df_isbn['Original_NaN'] = df_isbn['Age'].isna()

# Convert 'Age' to numeric, turning non-numeric values into NaN
df_isbn['Age'] = pd.to_numeric(df_isbn['Age'], errors='coerce')

# Drop rows where 'Age' is NaN and were not originally NaN
df_isbn = df_isbn[~(users['Age'].isna() & ~df_isbn['Original_NaN'])]

# Drop the 'Original_NaN' column as it's no longer needed
df_isbn.drop(columns=['Original_NaN'], inplace=True)

<ipython-input-11-a229782017b2>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_isbn = df_isbn[~(users['Age'].isna() & ~df_isbn['Original_NaN'])]
<ipython-input-11-a229782017b2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_isbn.drop(columns=['Original_NaN'], inplace=True)


In [ ]:
df_isbn['User-ID'] = pd.to_numeric(df_isbn['User-ID'], errors='coerce')

# Drop rows where 'Age' is NaN and were not originally NaN
df_isbn = df_isbn[~(df_isbn['User-ID'].isna())]

In [ ]:
age_outliers = df_isbn.query("Age > 100 or Age < 6")

user_outliers = age_outliers["User-ID"].to_list()

In [ ]:
# exclude age outliers
df_isbn = df_isbn[~df_isbn["User-ID"].isin(user_outliers)]

Embbedding fetching

In [ ]:
unique_descriptions = df_isbn['Description'].unique()

In [ ]:
from openai import OpenAI
client = OpenAI(api_key='key')

def get_embedding(text, model="text-embedding-3-small"):
    if text is None or pd.isna(text):
        return None
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [ ]:
description_to_embedding = {}

for description in tqdm(unique_descriptions, desc="Generating Embeddings"):
    if description not in description_to_embedding:
        embedding = get_embedding(description)
        description_to_embedding[description] = embedding

df_isbn['embedding'] = df_isbn['Description'].map(description_to_embedding)


Generating Embeddings: 100%|██████████| 38475/38475 [2:16:05<00:00,  4.71it/s]


In [ ]:
df_isbn['embedding'].notna().sum()

79833

In [ ]:
df_clened_with_emb = pd.read_csv('/content/df_with_emb_cleaned.csv')

In [ ]:
df_cleaned.rename(columns={"User-ID": "user"}, inplace=True)

<ipython-input-323-939667d10c5d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.rename(columns={"User-ID": "user"}, inplace=True)


In [ ]:
df_cleaned.rename(columns={"ISBN": "item"}, inplace=True)

<ipython-input-324-24d637dee9f4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.rename(columns={"ISBN": "item"}, inplace=True)


In [ ]:
df_cleaned.rename(columns={"Rating": "label"}, inplace=True)

<ipython-input-325-21c57fff3936>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.rename(columns={"Rating": "label"}, inplace=True)


In [ ]:
pca_columns = [f'pca_dim_{i+1}' for i in range(10)]

In [ ]:
columns = ['user', 'item', 'label']
user_cols = ['Age']
item_cols = pca_columns + ['Year']

columns = columns + user_cols + item_cols

## Training and Evaluation

In [ ]:
predictions = {}

In [ ]:
from libreco.algorithms import ALS, SVDpp
from sklearn.model_selection import train_test_split
from tensorflow.compat.v1 import reset_default_graph

from libreco.data import DataInfo, DatasetFeat, split_by_ratio
train_df, test_df = train_test_split(
    df_clened_with_emb,
    test_size=0.3,
    random_state=42
)

eval_user_mask = test_df['user'].isin(train_df['user'].unique())
eval_item_mask = test_df['item'].isin(train_df['item'].unique())

eval_df = test_df[eval_user_mask & eval_item_mask]

train_data, data_info = DatasetFeat.build_trainset(train_df, user_col=user_cols, item_col=item_cols, dense_col=item_cols + user_cols)
test_data = DatasetFeat.build_testset(eval_df)

reset_default_graph()
funkSVD = SVDpp(
    task="rating",
    data_info=data_info,
    n_epochs=4000,
    embed_size=256,
    lr=0.0005,
    reg=0.01,
    seed=42
)

funkSVD.fit(train_data, verbose=2, neg_sampling=False, eval_data=test_data, metrics=['rmse','mae'])

Training start time: 2024-08-11 15:00:27


train: 100%|██████████| 57/57 [00:00<00:00, 95.58it/s] 


Epoch 1 elapsed: 0.599s
	 train_loss: 66.7076


eval_pointwise: 100%|██████████| 1/1 [00:00<00:00, 624.62it/s]


	 eval rmse: 7.2591
	 eval mae: 7.0795


...

Epoch 3479 elapsed: 0.128s
	 train_loss: 29.1993


eval_pointwise: 100%|██████████| 1/1 [00:00<00:00, 524.62it/s]


	 eval rmse: 5.9371
	 eval mae: 5.4745


train:  33%|███▎      | 19/57 [00:00<00:00, 407.18it/s]


KeyboardInterrupt: 

In [ ]:
predictions = {}
for user_id in tqdm(eval_df['user'].unique()):
    predictions[user_id] = funkSVD.recommend_user(user=user_id, n_rec=20)[user_id]

100%|██████████| 273/273 [00:00<00:00, 650.27it/s]


In [ ]:
mean_ap = 0
total_abs_loss = 0
total_l2_loss = 0
total_precision_at_k = 0
total_recall_at_k = 0
total_ndcg = 0
total_dcg = 0

count_users_with_relevant_items = 0
count = 0

for user_id in eval_df['user'].unique():
    relevant_items = eval_df[eval_df['user']==user_id]['item'].to_numpy()

    predicted_isbns = predictions.get(user_id, [])

    if predicted_isbns is None:
        continue

    ap = average_precision(predicted_isbns, relevant_items, m=20)
    mean_ap += ap

    precision = precision_at_k(predicted_isbns, relevant_items, k=20)
    recall = recall_at_k(predicted_isbns, relevant_items, k=20)
    total_precision_at_k += precision
    total_recall_at_k += recall

    count_users_with_relevant_items += 1

mean_ap = mean_ap / count_users_with_relevant_items if count_users_with_relevant_items > 0 else 0
mean_precision_at_k = total_precision_at_k / count_users_with_relevant_items if count_users_with_relevant_items > 0 else 0
mean_recall_at_k = total_recall_at_k / count_users_with_relevant_items if count_users_with_relevant_items > 0 else 0

print(f"Mean Average Precision: {mean_ap}")
print(f"Mean Precision at K: {mean_precision_at_k}")
print(f"Mean Recall at K: {mean_recall_at_k}")

Mean Average Precision: 0.013829813893757753
Mean Precision at K: 0.0073260073260073295
Mean Recall at K: 0.07337218553773737
